In [81]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo
import attack
import scripts

import os
import matplotlib.pyplot as plt
import json

rand_seed=24
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

In [82]:
# purchase
# load data with different seeds
data_seed = {}
for rand_seed in [1,3,13,24,42]:
    data_seed[rand_seed] =  {}
    
    data_seed[rand_seed]['x_target_train'] = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
    data_seed[rand_seed]['y_target_train'] = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
    data_seed[rand_seed]['x_target_test'] = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
    data_seed[rand_seed]['y_target_test'] = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
    data_seed[rand_seed]['n_classes'] = len(np.unique(data_seed[rand_seed]['y_target_train']))
    data_seed[rand_seed]['X_train_size'] = data_seed[rand_seed]['x_target_train'].shape[0]
    data_seed[rand_seed]['X_test_size ']= data_seed[rand_seed]['x_target_test'].shape[0]

def set_the_seed_and_data(seed):
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    random.seed(rand_seed)
    
    return data_seed[seed]['x_target_train'], data_seed[rand_seed]['y_target_train'], data_seed[rand_seed]['x_target_test'], data_seed[rand_seed]['y_target_test']

In [83]:
# attack models
from torch import nn

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

# MI on centralized 

In [84]:
path = 'tm'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if "target_model_params.json" in file:
            with open(path+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
            
df = pd.DataFrame.from_dict(tms_params, orient='index')
df.shape            

(683, 18)

In [85]:
# attack every centralized target model
for file in tms_params:
    
    if 'attack_acc_mean' in tms_params[file]:
        continue
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_lr')])
    x_target_train, y_target_train, x_target_test, y_target_test = set_the_seed_and_data(rand_seed)
    
    #attack
    target_model = algo.LogisticRegression_DPSGD()
    target_model.theta = tms[file]
    params = tms_params[file]
    scripts.set_model_params(target_model, params)
    attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    
    params.update(attack_dict)
    if 'attack_acc' in params:
        params.pop('attack_acc')
        params.pop('attack_pre')
        params.pop('attack_rec')
    #write a new parameters file with attack results
    with open('tm/'+file+'_params.json', 'w') as file:
        json.dump(params, file)
        

# Membership inference Federated Learning


In [86]:
path = 'fl/'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:

#             with open(r+'/'+file, 'r') as f_open:
#                 lines = f_open.readlines()
#                 if 'HEAD' in lines[0]:   
#                     for i,l in enumerate(lines):
#                         if 'attack_acc_mean' in l:
#                             index = i
#                             break
#                     with open(r+'/'+file, 'w') as f_write:
#                         f_write.writelines(lines[i:-1])

            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            model_filenames = []
            if 'Unnamed: 0' in results[r].keys():
                for k in results[r]['Unnamed: 0']:
                    it.append(k[k.find('i')+1:k.find('_')])
                    client.append(k[k.find('_')+1:])
                    model_filenames.append(k+'.npy')
                results[r]['it'] = it
                results[r]['client'] = client
                results[r]['file_name'] = model_filenames
                results[r].pop('Unnamed: 0')
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
#             print(r+'/'+file)
            models[r][file] = np.load(r+'/'+file)



In [87]:
selected_files = results.keys()
len(selected_files)

286

In [88]:
# attack every federated local/global model
for file in selected_files:
    
    if 'attack_acc_mean' in results[file].keys():
        continue
        
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_ncl')])
    x_target_train, y_target_train, x_target_test, y_target_test = set_the_seed_and_data(rand_seed)
    #set number of client for following split of the training data between clients
    number_of_clients = len(results[file]['client'].unique())-1
    data_per_client = int(x_target_train.shape[0]/number_of_clients)

    #attack
    attack_results = {}
    print(file)
    for tm in results[file]['file_name']:
        target_model = algo.LogisticRegression_DPSGD()
        target_model.theta = models[file][tm]
        tm_params = params[file]
        scripts.set_model_params(target_model, tm_params)
        if 'g' in tm:
            target_model.x = x_target_train
            target_model.y = y_target_train
        else:
            i = int(tm[tm.find('_c')+2:tm.find('.npy')])
            target_model.x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            target_model.y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        attack_dict = attack.test_mi_attack(ams, target_model, target_model.x, target_model.y, x_target_test, y_target_test)
        attack_results[tm] = attack_dict
    attack_df = pd.DataFrame.from_dict(attack_results, orient='index')
    result_df = results[file].set_index('file_name')
    new_df = pd.merge(result_df, attack_df, left_index=True, right_index=True)
    #save attack results in old results file
    new_df.to_csv(file+'/results.csv')
    results[file] = new_df

In [89]:
new_df

,train_acc,test_acc,it,client,attack_acc_mean,attack_acc_std,attack_pre_mean,attack_pre_std,attack_rec_mean,attack_rec_std
file_name,,,,,,,,,,
i0_c0.npy,0.9560,0.4752,0,c0,0.649382,0.001709,0.606912,0.001032,0.848000,0.005608
i0_c1.npy,0.9472,0.4720,0,c1,0.637727,0.001357,0.597360,0.001094,0.845055,0.002746
i0_c2.npy,0.9604,0.4657,0,c2,0.646345,0.001455,0.603127,0.000954,0.855891,0.003967
i0_c3.npy,0.9508,0.4680,0,c3,0.654491,0.002119,0.614122,0.001236,0.831345,0.005616
i0_g.npy,0.6429,0.5198,0,g,0.548545,0.001124,0.537429,0.000717,0.697009,0.004674
i1_c0.npy,0.9724,0.5173,1,c0,0.620400,0.001424,0.582865,0.000802,0.846873,0.005010
i1_c1.npy,0.9736,0.5149,1,c1,0.618327,0.001106,0.581127,0.000715,0.847600,0.003094
i1_c2.npy,0.9804,0.5161,1,c2,0.622927,0.001263,0.583254,0.000832,0.861200,0.004538
i1_c3.npy,0.9748,0.5123,1,c3,0.629636,0.002326,0.591251,0.001255,0.839927,0.005881


# MI LOAN

In [90]:
path = 'mia/loan/best_ams/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('_params.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=14)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [91]:
path = 'loan/centr/'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
                    

In [92]:
# attack every centralized target model
for file in tms_params:
    
    if 'attack_acc_mean' in tms_params[file]:
        continue
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_lr')])
    x_target_train, y_target_train, x_target_test, y_target_test = scripts.load_loan(rand_seed, tr_size=10000)
    
    #attack
    target_model = algo.LogisticRegression_DPSGD()
    target_model.theta = tms[file]
    params = tms_params[file]
    scripts.set_model_params(target_model, params)
    attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    
    params.update(attack_dict)
    if 'attack_acc' in params:
        params.pop('attack_acc')
        params.pop('attack_pre')
        params.pop('attack_rec')
    #write a new parameters file with attack results
    with open('loan/centr/'+file+'_params.json', 'w') as file:
        json.dump(params, file)
        

# Loan FL

In [93]:
path = 'loan/fl'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            model_filenames = []
            if 'Unnamed: 0' in results[r].keys():
                for k in results[r]['Unnamed: 0']:
#                     print(k)
#                     break
                    it.append(k[k.find('i')+1:k.find('_')])
                    client.append(k[k.find('_')+1:])
                    model_filenames.append(k+'.npy')
                results[r]['it'] = it
                results[r]['client'] = client
                results[r]['file_name'] = model_filenames
                results[r].pop('Unnamed: 0')
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)



In [94]:
path = 'mia/loan/best_ams/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('_params.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=14)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [95]:
selected_files = results.keys()
len(selected_files)

591

In [96]:
# attack every federated local/global model
for file in selected_files:
    
    if 'attack_acc_mean' in results[file].keys():
        continue
        
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_ncl')])
    x_target_train, y_target_train, x_target_test, y_target_test = scripts.load_loan(rand_seed, tr_size=10000)
    #set number of client for following split of the training data between clients
    number_of_clients = len(results[file]['client'].unique())-1
    data_per_client = int(x_target_train.shape[0]/number_of_clients)

    #attack
    attack_results = {}
    print(file)
    for tm in results[file]['file_name']:
        target_model = algo.LogisticRegression_DPSGD()
        target_model.theta = models[file][tm]
        tm_params = params[file]
        scripts.set_model_params(target_model, tm_params)
        if 'g' in tm:
            target_model.x = x_target_train
            target_model.y = y_target_train
        else:
            i = int(tm[tm.find('_c')+2:tm.find('.npy')])
            target_model.x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            target_model.y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        attack_dict = attack.test_mi_attack(ams, target_model, target_model.x, target_model.y, x_target_test, y_target_test)
        attack_results[tm] = attack_dict
    attack_df = pd.DataFrame.from_dict(attack_results, orient='index')
    result_df = results[file].set_index('file_name')
    new_df = pd.merge(result_df, attack_df, left_index=True, right_index=True)
    #save attack results in old results file
    new_df.to_csv(file+'/results.csv')
    results[file] = new_df

# Texas

In [97]:
import os
path = 'mia/texas/best_mia/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=100)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [98]:
path = 'texas/centr/'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
                    

In [99]:
# attack every centralized target model
for file in tms_params:
    
    if 'attack_acc_mean' in tms_params[file]:
        continue
    # set random seed and load the data
    print(file)
    rand_seed=int(file[file.find('rs')+2:file.find('_lr')])
    x_target_train, y_target_train, x_target_test, y_target_test = scripts.load_texas()
    
    #attack
    target_model = algo.LogisticRegression_DPSGD()
    target_model.theta = tms[file]
    params = tms_params[file]
    scripts.set_model_params(target_model, params)
    attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    
    params.update(attack_dict)
    if 'attack_acc' in params:
        params.pop('attack_acc')
        params.pop('attack_pre')
        params.pop('attack_rec')
    #write a new parameters file with attack results
    with open('texas/centr/'+file+'_params.json', 'w') as file:
        json.dump(params, file)
        

# Texas FL 

In [100]:
path = 'texas/fl'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            model_filenames = []
            if 'Unnamed: 0' in results[r].keys():
                for k in results[r]['Unnamed: 0']:
#                     print(k)
#                     break
                    it.append(k[k.find('i')+1:k.find('_')])
                    client.append(k[k.find('_')+1:])
                    model_filenames.append(k+'.npy')
                results[r]['it'] = it
                results[r]['client'] = client
                results[r]['file_name'] = model_filenames
                results[r].pop('Unnamed: 0')
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)

path = 'mia/texas/best_mia/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=100)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [101]:
selected_files = results.keys()
len(selected_files)

4

In [102]:
# attack every federated local/global model
for file in selected_files:
    
    if 'attack_acc_mean' in results[file].keys():
        continue
        
    # set random seed and load the data    
    x_target_train, y_target_train, x_target_test, y_target_test = scripts.load_texas()
    number_of_clients = len(results[file]['client'].unique())-1
    data_per_client = int(x_target_train.shape[0]/number_of_clients)

    #attack
    attack_results = {}
    print(file)
    for tm in results[file]['file_name']:
        target_model = algo.LogisticRegression_DPSGD()
        target_model.theta = models[file][tm]
        tm_params = params[file]
        scripts.set_model_params(target_model, tm_params)
        if 'g' in tm:
            target_model.x = x_target_train
            target_model.y = y_target_train
        else:
            i = int(tm[tm.find('_c')+2:tm.find('.npy')])
            target_model.x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            target_model.y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        attack_dict = attack.test_mi_attack(ams, target_model, target_model.x, target_model.y, x_target_test, y_target_test)
        attack_results[tm] = attack_dict
    attack_df = pd.DataFrame.from_dict(attack_results, orient='index')
    result_df = results[file].set_index('file_name')
    new_df = pd.merge(result_df, attack_df, left_index=True, right_index=True)
    #save attack results in old results file
    new_df.to_csv(file+'/results.csv')
    results[file] = new_df

In [103]:
new_df

,train_acc,test_acc,it,client,attack_acc_mean,attack_acc_std,attack_pre_mean,attack_pre_std,attack_rec_mean,attack_rec_std
file_name,,,,,,,,,,
i0_c0.npy,0.9560,0.4752,0,c0,0.649382,0.001709,0.606912,0.001032,0.848000,0.005608
i0_c1.npy,0.9472,0.4720,0,c1,0.637727,0.001357,0.597360,0.001094,0.845055,0.002746
i0_c2.npy,0.9604,0.4657,0,c2,0.646345,0.001455,0.603127,0.000954,0.855891,0.003967
i0_c3.npy,0.9508,0.4680,0,c3,0.654491,0.002119,0.614122,0.001236,0.831345,0.005616
i0_g.npy,0.6429,0.5198,0,g,0.548545,0.001124,0.537429,0.000717,0.697009,0.004674
i1_c0.npy,0.9724,0.5173,1,c0,0.620400,0.001424,0.582865,0.000802,0.846873,0.005010
i1_c1.npy,0.9736,0.5149,1,c1,0.618327,0.001106,0.581127,0.000715,0.847600,0.003094
i1_c2.npy,0.9804,0.5161,1,c2,0.622927,0.001263,0.583254,0.000832,0.861200,0.004538
i1_c3.npy,0.9748,0.5123,1,c3,0.629636,0.002326,0.591251,0.001255,0.839927,0.005881
